In [7]:
class SnakeGameClass:
    def __init__(self,pathFood):
        self.initializeData()
        self.gameOver = False
        self.score = 0
        
        # Food interaction
        self.imgFood = cv2.imread(pathFood, cv2.IMREAD_UNCHANGED)
        self.hFood, self.wFood, _ = self.imgFood.shape # Image dimension
        
    def randomLocation(self):
        self.foodPoint = random.randint(100, 1000), random.randint(100,500)

    def initializeData(self):
        self.points = [] # Snake points
        self.lenghts = [] # Distance between points
        self.currentLength = 0 # Snake length
        self.allowedLength = 150
        self.previousHead = 0,0
        self.foodPoint = 0,0
        self.randomLocation()
        
    def update(self, imgMain, currentHead):
        font = cv2.FONT_HERSHEY_COMPLEX

        if self.gameOver:
            cv2.putText(imgMain, "Game Over", (50, 80), font, 2, (0, 204, 102), 4)
            cv2.putText(imgMain,f'Your Score {self.score}', (50, 140), font, 2, (0, 204, 102), 4)
        else:
            cx, cy = currentHead
            px, py = self.previousHead
            
            self.points.append((cx, cy))
            distance =  math.hypot(cx - px, cy - py)
            
            self.lenghts.append(distance)
            self.currentLength += distance
            self.previousHead = cx, cy
            self.reduceLength() # Length reduction

            # Check if the snake ate the food
            rx, ry = self.foodPoint
            if(rx-self.wFood//2 < cx < rx + self.wFood//2 and ry - self.hFood//2 < cy < ry + self.hFood//2):
                self.score += 1                
                self.allowedLength += 50
                self.randomLocation()
            
            # Draw the snake
            if(self.points):
                for i, point in enumerate(self.points):
                    if i != 0:
                        cv2.line(imgMain, self.points[i - 1], self.points[i], (0, 0, 255), 15)
                cv2.circle(imgMain, self.points[-1], 15, (0, 0, 153), cv2.FILLED)
            
            # Draw food
            cv2.putText(imgMain, f'Score {self.score}', (50, 80), font, 2, (0, 204, 102), 5)
            imgMain = cvzone.overlayPNG(imgMain, self.imgFood, (rx - self.wFood//2, ry - self.hFood//2))
            
            # Check for collision
            pts = np.array(self.points[:-2], np.int32)
            pts = pts.reshape((-1, 1, 2))
            
            cv2.polylines(imgMain, [pts], False, (0, 200, 0), 2)
            
            distace = cv2.pointPolygonTest(pts, (cx, cy), True)
            if(-1 <= distace <= 1):
                self.gameOver = True
                self.initializeData()
               
        return imgMain
                
    def reduceLength(self)  :
        if(self.currentLength > self.allowedLength):
            for i, lenght in enumerate(self.lenghts):
                # Keep reducing snake's length until it reaches the allowed length
                self.currentLength -= lenght
                self.lenghts.pop(i)
                self.points.pop(i)

                if(self.currentLength < self.allowedLength):
                    break

def play_snake_game(food_img):
    cap = cv2.VideoCapture(0)
    cap.set(3,1280)
    cap.set(4,1024)

    detector = HandDetector(detectionCon = 0.8, maxHands = 1)
    game = SnakeGameClass(food_img)

    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)
        
        hands, img = detector.findHands(img, flipType  = False, draw=False)
        if hands: # If a hand is detected
            lmList = hands[0]["lmList"]
            pointIndex = lmList[8][0:2]
            img = game.update(img, pointIndex)  # Update score & snake's length
        
        cv2.imshow('Snake Game', img)

        key = cv2.waitKey(1)    
        if key == ord('r'):
            game.gameOver = False
            game.score = 0
        if key ==  ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()